In [3]:
#! python -m spacy download en_core_web_sm

import spacy
from spacy.training.example import Example #Müssen später vermutlich was anderes nehmen
from spacy.training.iob_utils import offsets_to_biluo_tags
import random
import os
from training_set import training_data_got #sehr viele Warnings

# Load the base model
#nlp = spacy.blank("en")
nlp = spacy.load("en_core_web_sm")

# Create and add the NER pipeline component
ner = nlp.get_pipe("ner")

for text, annotations in training_data_got[0:100]:
    doc = nlp.make_doc(text)
    #biluo_tags = offsets_to_biluo_tags(doc, annotations["entities"]) # Mit Svens Daten ging es nicht

    # Initialize the optimizer
    optimizer = nlp.create_optimizer()

# Perform the training
for iteration in range(30):  #10 für die kleine Anzahl. Ansonsten könnte man ja so lange iterieren, bis die Fehler eine gewisse Untergrenze erreicht haben
    random.shuffle(training_data_got)
    losses = {}
    for text, annotations in training_data_got:
        doc = nlp.make_doc(text)
        example = Example.from_dict(doc, annotations)
        nlp.update([example], losses=losses, sgd=optimizer)
    #print("Losses:", losses)

# Save the trained NER model
#output_dir = os.getcwd()
output_dir = "C:\\Users\\Adriana\\Desktop\\Uni\\Fachpraktikum\\Repos\\concept-maps\\Adriana\\NER_NEU"
nlp.to_disk(output_dir)

# Load the saved model
loaded_nlp = spacy.load(output_dir)


In [4]:
import spacy
from spacy import displacy
import os
output_dir = os.getcwd()
loaded_nlp = spacy.load(output_dir)
text = "Sansa Stark is the daughter of Eddard Stark and Catelyn Stark"
# Test the loaded NER model
doc = loaded_nlp(text)
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
        token.shape_, token.is_alpha, token.is_stop)
displacy.serve(doc, style="ent")



Sansa Stark PERSON
Eddard Stark PERSON
Catelyn Stark PERSON
Sansa Sansa PROPN NNP compound Xxxxx True False
Stark Stark PROPN NNP nsubj Xxxxx True False
is be AUX VBZ ROOT xx True True
the the DET DT det xxx True True
daughter daughter NOUN NN attr xxxx True False
of of ADP IN prep xx True True
Eddard Eddard PROPN NNP compound Xxxxx True False
Stark Stark PROPN NNP pobj Xxxxx True False
and and CCONJ CC cc xxx True True
Catelyn Catelyn PROPN NNP compound Xxxxx True False
Stark Stark PROPN NNP conj Xxxxx True False



Using the 'ent' visualizer
Serving on http://0.0.0.0:5000 ...



In [ ]:
token = doc.ents
t1 = token[0].text
print(token[0])


Sansa Stark


In [ ]:
import json

# Laden der validierten Daten aus dem JSON-Dateiformat
with open('relationships.json', 'r') as file:
    validation_data = json.load(file)

# Ausgabe des "data" Objekts
print(validation_data)

# Iteriere über die Sätze im Validierungsdatensatz

for data in validation_data:
    sentence = data['sentence']
    entities = data['relationships']
    predicted_relationships=[]
    for entity in entities:
        firstEntity = entity['firstEntity']
        #relationshipType = entity['relationshipType'] # bei ner ja eignetlich unnötig
        secondEntity = entity['secondEntity']
        predicted_relationships.append(firstEntity)
        predicted_relationships.append(secondEntity)
    predicted_relationships = set(predicted_relationships)
    print(predicted_relationships)
    # Verarbeite den Satz mit dem geladenen NER-Modell
    doc = loaded_nlp(sentence)

    # Vergleiche die vorhergesagten Entitäten mit den tatsächlichen Entitäten
    predicted_entities = [(ent.text, ent.label_) for ent in doc.ents] # wird leer sein (war oben schon leer) , benötigen einen größeren Datensatz
    if predicted_entities == entities: # geht vermutlich bei arrays nicht
        print("Entitäten: Vorhersage stimmt überein")
    else:
        print("Entitäten: Vorhersage stimmt nicht überein")



[{'sentence': 'Queen Sansa Stark is the eldest daughter of Lord Eddard Stark and his wife, Lady Catelyn, sister of Robb, Arya, Bran, and Rickon Stark, and "half-sister" of Jon Snow; though truthfully is his cousin', 'relationships': [{'firstEntity': 'Sansa Stark', 'relationshipType': 'Father', 'secondEntity': 'Eddard Stark'}, {'firstEntity': 'Sansa Stark', 'relationshipType': 'Siblings', 'secondEntity': 'Jon Snow'}, {'firstEntity': 'Sansa Stark', 'relationshipType': 'Siblings', 'secondEntity': 'Rickon Stark'}, {'firstEntity': 'Jon Snow', 'relationshipType': 'Father', 'secondEntity': 'Eddard Stark'}, {'firstEntity': 'Eddard Stark', 'relationshipType': 'Children', 'secondEntity': 'Jon Snow'}, {'firstEntity': 'Eddard Stark', 'relationshipType': 'Children', 'secondEntity': 'Sansa Stark'}, {'firstEntity': 'Eddard Stark', 'relationshipType': 'Children', 'secondEntity': 'Rickon Stark'}, {'firstEntity': 'Rickon Stark', 'relationshipType': 'Father', 'secondEntity': 'Eddard Stark'}, {'firstEntit